In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Importing the necessary libraries for Colpali

In [2]:
!pip install -q colpali_engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 8.0 MB/s eta 0:00:000:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is inc

In [32]:
from colpali_engine.models import ColPali
from colpali_engine.models.paligemma.colpali.processing_colpali import ColPaliProcessor
from colpali_engine.utils.processing_utils import BaseVisualRetrieverProcessor
from colpali_engine.utils.torch_utils import ListDataset,get_torch_device
from torch.utils.data import DataLoader

import torch
from typing import List,cast
from tqdm import tqdm
from PIL import Image

# import spaces
import warnings
warnings.filterwarnings('ignore')

# 2. Initialting the model

In [4]:
model_name='vidore/colpali-v1.2'
device=get_torch_device('cuda')
print(device)

cuda


In [5]:
model=ColPali.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map=device
).eval()

processor=cast(ColPaliProcessor,ColPaliProcessor.from_pretrained(model_name))

adapter_config.json:   0%|          | 0.00/750 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/862M [00:00<?, ?B/s]

...

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

# 3. Converting all the PDF to images and storing to a folder

In [6]:
!apt-get update
!apt-get install -y poppler-utils

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]                             
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]                           
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]                           
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]                
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]                                
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]              
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,200 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubunt

In [7]:
from pdf2image import convert_from_path
from tqdm import tqdm
import shutil
import os

## 3.1 Delete the pdf_images folder if exists and then create a new folder

In [8]:
output_folder='/kaggle/working/page_images'
print(f"Cleaning output folder {output_folder}")
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.makedirs(output_folder)

Cleaning output folder /kaggle/working/page_images


## 3.2 Convert all the PDf pages to images and store in pdf_images folder

In [9]:
pdf_folder='/kaggle/input/prusa-mks3-5-pdf-data'

pdf_files=[file for file in os.listdir(pdf_folder) if file.lower().endswith('.pdf')]
len(pdf_files)

299

In [10]:
for pdf_file in tqdm(pdf_files,desc='Processing PDF files'):
    pdf_path=os.path.join(pdf_folder,pdf_file)
    pdf_name=os.path.splitext(pdf_file)[0]
    pdf_output_folder=os.path.join(output_folder,pdf_name)
    os.makedirs(pdf_output_folder,exist_ok=True)
    
    
    try:
        images=convert_from_path(pdf_path)
        for i,image in enumerate(images):
            image_path=os.path.join(pdf_output_folder,f"page_{i+1}.png")
            image.save(image_path,"PNG")
    except Exception as e:
        print(f"Failed to convert {pdf_file}:{e}")

Processing PDF files: 100%|██████████| 299/299 [02:47<00:00,  1.79it/s]


# 4. Create the embeddings for the PDF page images

In [11]:
!pip -q install spaces

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 7.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 99.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.0 MB/s eta 0:00:00


In [12]:
import spaces
from typing import List
from PIL import Image

In [30]:
@spaces.GPU
def get_images(folder_path: str) -> List[Image.Image]:
    images = []
    folders = os.listdir(folder_path)
    
    for folder in folders:
        subfolder_path = os.path.join(folder_path, folder)
        if os.path.isdir(subfolder_path):
            for file in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, file)
                try:
                    img = Image.open(file_path)
                    images.append(img)
                except Exception as e:
                    print(f"Skipping {file_path}: {e}")
        else:
            try:
                img = Image.open(subfolder_path)
                images.append(img)
            except Exception as e:
                print(f"Skipping {subfolder_path}: {e}")
    
    return images

image_list=get_images('/kaggle/working/page_images')
len(image_list)

458

In [33]:
datalaoder=DataLoader(
    dataset=ListDataset[str](image_list),
    batch_size=10,
    shuffle=False,
    collate_fn=lambda x:processor.process_images(x),
)

In [34]:
ds:List[torch.Tensor]=[]
for batch_doc in tqdm(datalaoder):
    with torch.no_grad():
        batch_doc={k:v.to(model.device) for k,v in batch_doc.items()}
        embeddings_doc=model(**batch_doc)
    ds.extend(list(torch.unbind(embeddings_doc.to(device))))
ds_np=[d.float().cpu().numpy() for d in ds]
ds_np

 41%|████▏     | 19/46 [09:31<13:32, 30.08s/it]


KeyboardInterrupt: 